In [1]:
import pandas as pd

# create simple transaction dataset
data = {
    "CustomerID": [1,1,2,2,3,4,5,5,6,7,8,9,10,10],
    "InvoiceDate": [
        "2024-01-01","2024-02-10","2024-02-01","2024-03-01",
        "2024-03-15","2024-01-20","2024-02-25","2024-03-05",
        "2024-01-10","2024-03-18","2024-02-14","2024-03-02",
        "2024-01-05","2024-03-12"
    ],
    "Amount": [100,150,200,50,300,120,80,60,500,40,220,90,130,170]
}

df = pd.DataFrame(data)

# convert InvoiceDate to datetime
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

df


,CustomerID,InvoiceDate,Amount
0,1,2024-01-01,100
1,1,2024-02-10,150
2,2,2024-02-01,200
3,2,2024-03-01,50
4,3,2024-03-15,300
5,4,2024-01-20,120
6,5,2024-02-25,80
7,5,2024-03-05,60
8,6,2024-01-10,500
9,7,2024-03-18,40


In [4]:
reference_date = df["InvoiceDate"].max()+pd.Timedelta(days=1)

rfm=df.groupby("CustomerID").agg({
    "InvoiceDate": lambda x: (reference_date-x.max()).days,
    "CustomerID": "count",
    "Amount": "sum"
})
rfm.columns=["Recency","Frequency","Monetary"]
rfm

,Recency,Frequency,Monetary
CustomerID,,,
1,38,2,250
2,18,2,250
3,4,1,300
4,59,1,120
5,14,2,140
6,69,1,500
7,1,1,40
8,34,1,220
9,17,1,90


In [5]:
# create RFM score buckets using quantiles
rfm["R_score"] = pd.qcut(rfm["Recency"], 3, labels=[3,2,1])
rfm["F_score"] = pd.qcut(rfm["Frequency"].rank(method="first"), 3, labels=[1,2,3])
rfm["M_score"] = pd.qcut(rfm["Monetary"], 3, labels=[1,2,3])

# combine scores
rfm["RFM_Score"] = rfm["R_score"].astype(str) + rfm["F_score"].astype(str) + rfm["M_score"].astype(str)

# assign simple segment labels
def segment_label(score):
    if score == "333":
        return "Champions"
    elif score[0] == "1":
        return "At Risk"
    else:
        return "Loyal"

rfm["Segment"] = rfm["RFM_Score"].apply(segment_label)

rfm


,Recency,Frequency,Monetary,R_score,F_score,M_score,RFM_Score,Segment
CustomerID,,,,,,,,
1,38,2,250,1,3,2,132,At Risk
2,18,2,250,2,3,2,232,Loyal
3,4,1,300,3,1,3,313,Loyal
4,59,1,120,1,1,1,111,At Risk
5,14,2,140,2,3,2,232,Loyal
6,69,1,500,1,1,3,113,At Risk
7,1,1,40,3,2,1,321,Loyal
8,34,1,220,1,2,2,122,At Risk
9,17,1,90,2,2,1,221,Loyal


In [6]:
# export RFM segmentation table
rfm.to_csv("rfm_segments.csv")

# create business actions text
actions = """
Champions:
- Offer loyalty rewards and exclusive discounts.
- Encourage referrals and premium memberships.
- Provide early access to new products.

Loyal Customers:
- Send personalized offers to increase purchase frequency.
- Promote bundle deals and cross-selling.
- Maintain engagement through regular communication.

At Risk Customers:
- Send re-engagement emails with special discounts.
- Offer limited-time promotions to win them back.
- Collect feedback to understand churn reasons.
"""

with open("segment_actions.txt", "w") as f:
    f.write(actions)

print("Files created: rfm_segments.csv and segment_actions.txt")


Files created: rfm_segments.csv and segment_actions.txt
